In [1]:
import json
import os

import networkx as nx
with open("actors_projection_graph.json") as f:
    graph_dict = json.load(f)

graph = nx.node_link_graph(graph_dict)

In [2]:
with open("successful_actors.json") as f:
    successful = json.load(f)
with open("almost_successful_actors.json") as f:
    almost_successful = json.load(f)

In [3]:
print(len(successful))
print(len(almost_successful))

453
14892


In [5]:
print(successful["actor: Tom Hanks"])

{'gender': 'male', 'countries': ['Hungary', 'Germany', 'India', 'Mexico', 'United States of America', 'France', 'United Kingdom', 'Singapore', 'Australia', 'Italy', 'Hong Kong', 'Malta'], 'avg_movie_revenue_2020_$': 97984814, 'top_ten': [[1995, 'movie: Toy Story'], [1995, 'movie: Apollo 13'], [1994, 'movie: Forrest Gump'], [1993, 'movie: Philadelphia'], [1993, 'movie: Sleepless in Seattle'], [1996, 'movie: That Thing You Do!'], [1998, 'movie: Saving Private Ryan'], [1989, "movie: The 'Burbs"], [1984, 'movie: Splash'], [1986, 'movie: The Money Pit'], [1986, 'movie: Nothing in Common'], [1998, "movie: You've Got Mail"], [1988, 'movie: Big'], [1999, 'movie: Return with Honor'], [1999, 'movie: Toy Story 2'], [1990, 'movie: The Bonfire of the Vanities'], [1999, 'movie: The Green Mile'], [1992, 'movie: A League of Their Own'], [1985, 'movie: Volunteers'], [1984, 'movie: Bachelor Party'], [1988, 'movie: Punchline'], [2000, 'movie: Cast Away'], [1989, 'movie: Turner & Hooch'], [1980, "movie: H

In [3]:
nodes = [(n,d) for n, d in graph.nodes(data=True)]
edges = [(e1,e2,d) for e1,e2, d in graph.edges(data=True)]


In [4]:
print(edges[0])
print(nodes[0])

('actor: Tom Hanks', 'actor: Kamron Leal', {'weight': 1, 'movies': [['movie: Sully', 2016]], 'earliest_contact': 2016})
('actor: Tom Hanks', {'gender': 'male', 'countries': ['Hungary', 'Germany', 'India', 'Mexico', 'United States of America', 'France', 'United Kingdom', 'Singapore', 'Australia', 'Italy', 'Hong Kong', 'Malta'], 'avg_movie_revenue_2020_$': 97984814, 'top_ten': [[1995, 'movie: Toy Story'], [1995, 'movie: Apollo 13'], [1994, 'movie: Forrest Gump'], [1993, 'movie: Philadelphia'], [1993, 'movie: Sleepless in Seattle'], [1996, 'movie: That Thing You Do!'], [1998, 'movie: Saving Private Ryan'], [1989, "movie: The 'Burbs"], [1984, 'movie: Splash'], [1986, 'movie: The Money Pit'], [1986, 'movie: Nothing in Common'], [1998, "movie: You've Got Mail"], [1988, 'movie: Big'], [1999, 'movie: Return with Honor'], [1999, 'movie: Toy Story 2'], [1990, 'movie: The Bonfire of the Vanities'], [1999, 'movie: The Green Mile'], [1992, 'movie: A League of Their Own'], [1985, 'movie: Volunteers'

In [4]:
# let's create a graph with only the most successful actors to see how they connect to each other

niche_graph = nx.DiGraph()
for actor,diz in successful.items():
    niche_graph.add_node(actor, **diz)

for edge in edges:
    actor1 = edge[0]
    actor2 = edge[1]
    if actor1 in successful and actor2 in successful:
        if "contacts_before_breakthrough" in graph.nodes[actor1]:
            for colleague in graph.nodes[actor1]["contacts_before_breakthrough"]:
                if actor2 == colleague[0]:
                    niche_graph.add_edge(actor1, actor2, **{"year":colleague[1]})
        if "contacts_before_breakthrough" in graph.nodes[actor2]:
            for colleague in graph.nodes[actor2]["contacts_before_breakthrough"]:
                if actor1 == colleague[0]:
                    niche_graph.add_edge(actor2, actor1, **{"year":colleague[1]})

to_dump = nx.readwrite.json_graph.node_link_data(niche_graph)
with open("niche_graph.json", "w") as f:
    json.dump(to_dump, f, indent = 4)



In [5]:
# let's create a graph with only non-successful actors to see how they connect to each other

almost_niche_graph = nx.DiGraph()
for actor,diz in almost_successful.items():
    almost_niche_graph.add_node(actor, **diz)

for edge in edges:
    actor1 = edge[0]
    actor2 = edge[1]
    if actor1 in almost_successful and actor2 in almost_successful:
        if "contacts_before_breakthrough" in graph.nodes[actor1]:
            for colleague in graph.nodes[actor1]["contacts_before_breakthrough"]:
                if actor2 == colleague[0]:
                    almost_niche_graph.add_edge(actor1, actor2, **{"year":colleague[1]})
        if "contacts_before_breakthrough" in graph.nodes[actor2]:
            for colleague in graph.nodes[actor2]["contacts_before_breakthrough"]:
                if actor1 == colleague[0]:
                    almost_niche_graph.add_edge(actor2, actor1, **{"year":colleague[1]})


to_dump = nx.readwrite.json_graph.node_link_data(almost_niche_graph)
with open("almost_niche_graph.json", "w") as f:
    json.dump(to_dump, f, indent = 4)
print(len(almost_niche_graph))



14892


In [ ]:
# let's create a bipartite graph with the most successful actors and those who almost made it to see how they connect to each other


mixed_graph = nx.DiGraph()
for actor,diz in successful.items():
    diz["status"] = "successful"
    mixed_graph.add_node(actor, **diz, bipartite=0)
for actor,diz in almost_successful.items():
    diz["status"] = "almost_successful"
    mixed_graph.add_node(actor, **diz, bipartite =1)

for edge in edges:
    actor1 = edge[0]
    actor2 = edge[1]
    if actor1 in almost_successful and actor2 in successful:
        if "contacts_before_breakthrough" in graph.nodes[actor1]:
            for colleague in graph.nodes[actor1]["contacts_before_breakthrough"]:
                if actor2 == colleague[0]:
                    mixed_graph.add_edge(actor1, actor2, **{"year":colleague[1]})
        if "contacts_before_breakthrough" in graph.nodes[actor2]:
            for colleague in graph.nodes[actor2]["contacts_before_breakthrough"]:
                if actor1 == colleague[0]:
                    mixed_graph.add_edge(actor2, actor1, **{"year":colleague[1]})
    if actor1 in successful and actor2 in almost_successful:
        if "contacts_before_breakthrough" in graph.nodes[actor1]:
            for colleague in graph.nodes[actor1]["contacts_before_breakthrough"]:
                if actor2 == colleague[0]:
                    mixed_graph.add_edge(actor1, actor2, **{"year":colleague[1]})
        if "contacts_before_breakthrough" in graph.nodes[actor2]:
            for colleague in graph.nodes[actor2]["contacts_before_breakthrough"]:
                if actor1 == colleague[0]:
                    mixed_graph.add_edge(actor2, actor1, **{"year": colleague[1]})



to_dump = nx.readwrite.json_graph.node_link_data(mixed_graph)
with open("mixed_graph.json", "w") as f:
    json.dump(to_dump, f, indent = 4)

if nx.bipartite.is_bipartite(mixed_graph):
    print("Il grafo è bipartito correttamente.")
else:
    print("Errore: ci sono archi interni ai set!")

In [ ]:
# find mentors, mentees and peers
# mentor: someone who has an edge incoming from someone, but not one outgoing to that someone
# mentor: someone who has an edge outgoing to someone, but not one incoming from that someone
# peers: actors who knew each other before either made it


# ANALYSIS

### CENTRALITIES

In [24]:
import statistics
import pandas as pd

def compute_centrality_nx(kind, function,df):
    niche_betweenness = function(niche_graph)
    almost_niche_betweenness = function(almost_niche_graph)
    mixed_betweenness = function(mixed_graph)

    items_niche_betweenness = niche_betweenness.items()
    top_10_niche_betweenness = sorted(items_niche_betweenness, key = lambda x: x[1], reverse = True)[:10]
    avg_niche_betweenness = statistics.mean([value for name, value in niche_betweenness.items()])
    max_niche_betweenness = max([value for name,value in top_10_niche_betweenness])

    items_almost_niche_betweenness = almost_niche_betweenness.items()
    top_10_almost_niche_betweenness  = sorted(items_almost_niche_betweenness, key = lambda x: x[1], reverse = True)[:10]
    avg_almost_niche_betweenness= statistics.mean([value for name, value in almost_niche_betweenness.items()])
    max_almost_niche_betweenness = max([value for name,value in top_10_almost_niche_betweenness])

    items_mixed_betweenness= mixed_betweenness.items()
    top_10_mixed_betweenness  = sorted(items_mixed_betweenness, key = lambda x: x[1], reverse = True)[:10]
    avg_mixed_betweenness = statistics.mean([value for name, value in mixed_betweenness.items()])
    max_mixed_betweenness = max([value for name,value in top_10_mixed_betweenness])

    df.loc[f"Average {kind} centrality"] = [avg_niche_betweenness, avg_almost_niche_betweenness, avg_mixed_betweenness]
    df.loc[f"Maximum {kind} centrality"] = [max_niche_betweenness, max_almost_niche_betweenness, max_mixed_betweenness]


    to_save = dict()
    to_save["niche_graph"] = top_10_niche_betweenness
    to_save["almost_niche_graph"] = top_10_almost_niche_betweenness
    to_save["mixed_graph"] = top_10_mixed_betweenness

    # with open(f"analysis_5/{kind}_centralities.json", "w") as f:
    #     json.dump(to_save,f, indent=4)
centrality_df = pd.DataFrame(columns=["Niche graph", "Almost niche graph", "Mixed graph"])


In [ ]:
import pandas as pd
import networkit as nk
centrality_df = pd.DataFrame(columns=["Niche graph", "Almost niche graph", "Mixed graph"])
centrality_df = pd.read_json("analysis_5/centrality.json")
print(centrality_df)
niche_nodes_list = list(niche_graph.nodes())
almost_niche_nodes_list = list(almost_niche_graph.nodes())
mixed_nodes_list = list(mixed_graph.nodes())
nk.setNumberOfThreads(7)
niche_graph_nk = nk.nxadapter.nx2nk(niche_graph)
almost_niche_graph_nk = nk.nxadapter.nx2nk(almost_niche_graph)
mixed_graph_nk = nk.nxadapter.nx2nk(mixed_graph)

Empty DataFrame
Columns: [Niche graph, Almost niche graph, Mixed graph]
Index: []


In [14]:
import statistics
import pandas as pd
import networkit as nk
import os


top10_betweenness = dict()
avg_betweenness = dict()
max_betweenness = dict()

def compute_betweenness(graph, graph_name, nodes_list):
    betweenness = nk.centrality.Betweenness(graph, normalized=True)
    betweenness.run()
    scores = betweenness.scores()
    node_scores = list(enumerate(scores))
    #print(scores)
    id_sorted_scores = sorted(node_scores, key=lambda x: x[1], reverse=True)
    name_sorted_scores = list()
    for id, score in id_sorted_scores[:10]:
        name = nodes_list[id]
        name_sorted_scores.append((name, score))
    top_10_betweenness = name_sorted_scores
    avg_betweenness_value = statistics.mean(scores)
    max_betweenness_value = name_sorted_scores[0][1]
    top10_betweenness[graph_name] = top_10_betweenness
    avg_betweenness[graph_name] = avg_betweenness_value
    max_betweenness[graph_name] = max_betweenness_value

compute_betweenness(niche_graph_nk, "niche_graph", niche_nodes_list)
compute_betweenness(almost_niche_graph_nk, "almost_niche_graph", almost_niche_nodes_list)
compute_betweenness(mixed_graph_nk, "mixed_graph", mixed_nodes_list)

avg_betweenness_list = list(avg_betweenness.values())
max_betweenness_list = list(max_betweenness.values())

centrality_df.loc[f"Average betweenness centrality"] = avg_betweenness_list
centrality_df.loc[f"Maximum betweenness centrality"] = max_betweenness_list
to_save = dict()

to_save["niche_graph"] = top10_betweenness["niche_graph"]
to_save["almost_niche_graph"] = top10_betweenness["almost_niche_graph"]
to_save["mixed_graph"] = top10_betweenness["mixed_graph"]
folder_name = "analysis_5"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Created folder {folder_name}")
else:
    print(f"Folder {folder_name} already exists")
with open("analysis_5/betweenness_centralities.json", "w") as f:
    json.dump(to_save,f)

Folder analysis_5 already exists


In [15]:
print(centrality_df)

                                Niche graph  Almost niche graph  Mixed graph
Average betweenness centrality     0.004591            0.000092     0.000052
Maximum betweenness centrality     0.086162            0.022253     0.008766


In [25]:
folder_name = "analysis_5"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Created folder {folder_name}")
else:
    print(f"Folder {folder_name} already exists")
compute_centrality_nx("IN-degree", nx.in_degree_centrality, centrality_df)
compute_centrality_nx("OUT-degree", nx.out_degree_centrality, centrality_df)
print(centrality_df)
centrality_df.to_json("analysis_5/ndejf.json")

Folder analysis_5 already exists
                               Niche graph  Almost niche graph  Mixed graph
Average IN-degree centrality      0.022803            0.001980     0.000353
Maximum IN-degree centrality      0.075221            0.042710     0.021051
Average OUT-degree centrality     0.022803            0.001980     0.000353
Maximum OUT-degree centrality     0.137168            0.048284     0.026786


In [ ]:
import statistics
top10_closeness = dict()
avg_closeness = dict()
max_closeness = dict()
def compute_closeness(graph, graph_name, nodes_list):
    closeness= nk.centrality.HarmonicCloseness(graph)
    closeness.run()
    scores = closeness.scores()
    node_scores = list(enumerate(scores))
    id_sorted_scores = sorted(node_scores, key=lambda x: x[1], reverse=True)
    name_sorted_scores = list()
    for id, score in id_sorted_scores[:10]:
        name = nodes_list[id]
        name_sorted_scores.append((name, score))
    top_10= name_sorted_scores
    avg_value = statistics.mean(scores)
    max_value = name_sorted_scores[0][1]
    top10_closeness[graph_name] = top_10
    avg_closeness[graph_name] = avg_value
    max_closeness[graph_name] = max_value

compute_closeness(niche_graph_nk, "niche_graph", niche_nodes_list)
compute_closeness(almost_niche_graph_nk, "almost_niche_graph", almost_niche_nodes_list)
# nb check networkx for closeness 
compute_closeness(mixed_graph_nk, "mixed_graph", mixed_nodes_list)

avg_closeness_list = list(avg_closeness.values())
max_closeness_list = list(max_closeness.values())

centrality_df.loc[f"Average closeness centrality"] = avg_closeness_list
centrality_df.loc[f"Maximum closeness centrality"] = max_closeness_list
to_save = dict()

to_save["niche_graph"] = top10_closeness["niche_graph"]
to_save["almost_niche_graph"] = top10_closeness["almost_niche_graph"]
to_save["mixed_graph"] = top10_closeness["mixed_graph"]
folder_name = "analysis_5"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Created folder {folder_name}")
else:
    print(f"Folder {folder_name} already exists")
with open("analysis_5/closeness_centralities.json", "w") as f:
    json.dump(to_save,f)

Folder analysis_5 already exists


In [18]:
print(centrality_df)

                                Niche graph  Almost niche graph  Mixed graph
Average betweenness centrality     0.004591            0.000092     0.000052
Maximum betweenness centrality     0.086162            0.022253     0.008766
Average IN-degree centrality       0.022803            0.001980     0.000353
Maximum IN-degree centrality       0.002212            0.000000     0.000000
Average OUT-degree centrality      0.022803            0.001980     0.000353
Maximum OUT-degree centrality      0.002212            0.000000     0.000000
Average closeness centrality       0.254731            0.181684     0.064286
Maximum closeness centrality       0.517883            0.441322     0.186468


In [ ]:
import numpy as np
compute_centrality_nx("pagerank", nx.pagerank, centrality_df)
n_to_multiply = np.array([len(niche_nodes_list), len(almost_niche_nodes_list), len(mixed_nodes_list)])
max_pr = centrality_df.loc["Maximum pagerank centrality"].values
processed_max_pr = list(max_pr*n_to_multiply)
centrality_df.loc["Maximum normalized pagerank centrality"] = processed_max_pr

print(centrality_df)
centrality_df.to_json("analysis_5/centrality.json")


                                        Niche graph  Almost niche graph  \
Average pagerank centrality                0.002208            0.000067   
Maximum pagerank centrality                0.024040            0.001011   
Maximum normalized pagerank centrality    10.890214           15.054149   

                                        Mixed graph  
Average pagerank centrality                0.000065  
Maximum pagerank centrality                0.003866  
Maximum normalized pagerank centrality    59.320996  


### NETWORK STRUCTURE

In [20]:
import pandas as pd
print("Strongly-connected components - continuous collaboration")
niche_scc_sets = list(nx.strongly_connected_components(niche_graph))
niche_scc = len(max(niche_scc_sets, key=len))
almost_niche_scc_sets = list(nx.strongly_connected_components(almost_niche_graph))
almost_niche_scc = len(max(almost_niche_scc_sets, key=len))
mixed_scc_sets = list(nx.strongly_connected_components(mixed_graph))
mixed_scc = len(max(mixed_scc_sets, key=len))


structure_df = pd.DataFrame({"Niche graph":niche_scc, "Almost niche graph":almost_niche_scc, "Mixed graph": mixed_scc}, index = ["Biggest SCC size"])
structure_df.loc["Biggest SCC size ratio"] = [niche_scc/len(niche_nodes_list), almost_niche_scc/len(almost_niche_nodes_list), mixed_scc/len(mixed_nodes_list)]

print(structure_df)

Strongly-connected components - continuous collaboration
                        Niche graph  Almost niche graph  Mixed graph
Biggest SCC size         350.000000         9043.000000  5507.000000
Biggest SCC size ratio     0.772627            0.607239     0.358879


In [21]:
print("Weakly-connected components - isolated 'families'")
niche_wcc_sets = list(nx.weakly_connected_components(niche_graph))
niche_wcc = len(max(niche_wcc_sets, key=len))
almost_niche_wcc_sets = list(nx.weakly_connected_components(almost_niche_graph))
almost_niche_wcc = len(max(almost_niche_wcc_sets, key=len))
mixed_wcc_sets = list(nx.weakly_connected_components(mixed_graph))
mixed_wcc = len(max(mixed_wcc_sets, key=len))

structure_df.loc["Biggest WCC size"] = [niche_wcc, almost_niche_wcc, mixed_wcc]
structure_df.loc["Biggest WCC size ratio"] = [niche_wcc/len(niche_nodes_list), almost_niche_wcc/len(almost_niche_nodes_list), mixed_wcc/len(mixed_nodes_list)]
print(structure_df)


Weakly-connected components - isolated 'families'
                        Niche graph  Almost niche graph  Mixed graph
Biggest SCC size         350.000000         9043.000000  5507.000000
Biggest SCC size ratio     0.772627            0.607239     0.358879
Biggest WCC size         452.000000        14141.000000  9979.000000
Biggest WCC size ratio     0.997792            0.949570     0.650310


In [25]:
niche_density =nx.density(niche_graph)
almost_niche_density = nx.density(almost_niche_graph)
mixed_density = nx.density(mixed_graph)
structure_df.loc["Density"] = [niche_density, almost_niche_density, mixed_density]



In [26]:
print("Reciprocity - became famous together")
niche_reciprocity = nx.reciprocity(niche_graph)
almost_niche_reciprocity = nx.reciprocity(almost_niche_graph)
mixed_reciprocity = nx.reciprocity(mixed_graph)
structure_df.loc["Reciprocity"] = [niche_reciprocity, almost_niche_reciprocity, mixed_reciprocity]


Reciprocity - became famous together


In [ ]:
print("Transitivity")
# Probabilità che se A ha lavorato con B e B con C, anche A abbia lavorato con C.
niche_transitivity = nx.transitivity(niche_graph)
almost_niche_transitivity = nx.transitivity(almost_niche_graph)
mixed_transitivity = 0
structure_df.loc["Transitivity"] = [niche_transitivity, almost_niche_transitivity, mixed_transitivity]



Transitivity


In [ ]:
print("Clustering")
niche_clustering = nx.average_clustering(niche_graph)
almost_niche_clustering = nx.average_clustering(almost_niche_graph)
mixed_clustering = nx.bipartite.clustering(mixed_graph)
structure_df.loc["Clustering"] = [niche_clustering, almost_niche_clustering, mixed_clustering]


Clustering


In [29]:
structure_df.to_json("analysis_5/structure.json")

### ASSORTATIVITY

In [30]:
# degree assortativity
import pandas as pd
niche_degree_assort = nx.degree_assortativity_coefficient(niche_graph)
almost_niche_degree_assort = nx.degree_assortativity_coefficient(almost_niche_graph)
assortativity_df = pd.DataFrame({"Niche graph": niche_degree_assort, "Almost niche graph": almost_niche_degree_assort, "Mixed graph": None}, index = ["Degree assortativity"])


In [31]:
# attribute assortativity
import pandas as pd
mixed_attribute_assort = nx.attribute_assortativity_coefficient(mixed_graph, "status")
assortativity_df.loc["Attribute assortativity"] = [None, None, mixed_attribute_assort]
assortativity_df.to_json("analysis_5/assortativity.json")
mixed_mixing_matrix = nx.attribute_mixing_matrix(mixed_graph, "status")

/tmp/ipykernel_4176/2807361511.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  assortativity_df.loc["Attribute assortativity"] = [None, None, mixed_attribute_assort]


In [32]:
mixed_matrix_df = pd.DataFrame({"From almost-famous to famous actor": mixed_mixing_matrix[0,1], "From famous to almost-famous actor": mixed_mixing_matrix[1,0]}, index = ["Collaborations"])
print(mixed_matrix_df)
mixed_matrix_df.to_json("analysis_5/mixed_matrix.json")

                From almost-famous to famous actor  \
Collaborations                            0.365698   

                From famous to almost-famous actor  
Collaborations                            0.634302  


### DISTANCE METRICS

In [33]:
import numpy as np
import random
import math
import pandas as pd
import networkit as nk
distance_df = pd.DataFrame(columns=["Niche graph", "Almost niche graph", "Mixed graph"])
diameter_list= list()
avg_sp = list()
def small_world_analysis(main_component, graph_name):
    print(f"Working on {graph_name}")
    main_component = nk.nxadapter.nx2nk(main_component)
    print("Main component number of nodes:", main_component.numberOfNodes())
    print("Main component number of edges:", main_component.numberOfEdges())
    print("Analysing the diameter...")
    diameter = nk.distance.Diameter(main_component, algo=nk.distance.DiameterAlgo.EXACT)
    diameter.run()
    diameter_value = diameter.getDiameter()
    print("Diameter:", diameter_value[0])
    diameter_list.append(diameter_value[0])
    print("Computing the average path length (sampled)...")
    all_nodes = list(main_component.iterNodes())
    sampled_nodes = all_nodes
    batch_size = 200
    num_batches = math.ceil(len(all_nodes) / batch_size)
    total_distance_sum = 0
    total_valid_pairs = 0
    print("Number of batches:", num_batches)
    #for node in main_component.iterNodes():
    for batch_iter in range(num_batches):
        start_idx = batch_iter * batch_size
        end_idx = min((batch_iter + 1) * batch_size, len(all_nodes))
        batch_nodes = sampled_nodes[start_idx:end_idx]

        spsp = nk.distance.SPSP(main_component, batch_nodes)
        spsp.run()
        distances_from_source = spsp.getDistances()

        distances_array = np.array(distances_from_source)
        valid_mask = distances_array > 0
        total_distance_sum += distances_array[valid_mask].sum()
        total_valid_pairs += valid_mask.sum()

        del spsp
        del distances_array
        if batch_iter % 10 == 0 and batch_iter != 0:
            print("Finished batch:", batch_iter)

    avg_path_length = total_distance_sum / total_valid_pairs
    print("Average sampled path length:", avg_path_length)
    avg_sp.append(avg_path_length)



In [34]:
import networkx as nx
niche_main_cc = max(nx.weakly_connected_components(niche_graph), key = len)
niche_main_cc = niche_graph.subgraph(niche_main_cc).copy()
niche_main_cc = niche_main_cc.to_undirected()
almost_niche_main_cc = max(nx.weakly_connected_components(almost_niche_graph), key = len)
almost_niche_main_cc = almost_niche_graph.subgraph(almost_niche_main_cc).copy()
almost_niche_main_cc = almost_niche_main_cc.to_undirected()
mixed_main_cc = max(nx.weakly_connected_components(mixed_graph), key = len)
mixed_main_cc = mixed_graph.subgraph(mixed_main_cc).copy()
mixed_main_cc = mixed_main_cc.to_undirected()



In [35]:
small_world_analysis(niche_main_cc, "niche main component")
small_world_analysis(almost_niche_main_cc, "almost_niche main component")
small_world_analysis(mixed_main_cc, "mixed main component")

Working on niche main component
Main component number of nodes: 452
Main component number of edges: 3932
Analysing the diameter...
Diameter: 5
Computing the average path length (sampled)...
Number of batches: 3
Average sampled path length: 2.563840433255499
Working on almost_niche main component
Main component number of nodes: 14141
Main component number of edges: 346990
Analysing the diameter...
Diameter: 9
Computing the average path length (sampled)...
Number of batches: 71
Finished batch: 10
Finished batch: 20
Finished batch: 30
Finished batch: 40
Finished batch: 50
Finished batch: 60
Finished batch: 70
Average sampled path length: 3.0984554227392795
Working on mixed main component
Main component number of nodes: 9979
Main component number of edges: 69565
Analysing the diameter...
Diameter: 6
Computing the average path length (sampled)...
Number of batches: 50
Finished batch: 10
Finished batch: 20
Finished batch: 30
Finished batch: 40
Average sampled path length: 3.7540411131164584


In [36]:
distance_df.loc["Diameter"] = diameter_list
distance_df.loc["Main connected component shortest path"] = avg_sp




In [37]:
distance_df.loc["Graph - number of nodes"] = [len(list(niche_graph.nodes())), len(list(almost_niche_graph.nodes())), len(list(mixed_graph.nodes()))]
distance_df.loc["Graph - number of edges"] = [len(list(niche_graph.edges())), len(list(almost_niche_graph.edges())), len(list(mixed_graph.edges()))]
distance_df.loc["Number of connected components"] = [len(niche_wcc_sets), len(almost_niche_wcc_sets), len(mixed_wcc_sets)]
distance_df.to_json("analysis_5/distance.json")